In [3]:
import csv
import random
import math
import operator
#1、处理数据 首先，我们要把加载进来的特征数据从字符串转换成整数。
#然后我们随机地切分训练数据集和测试数据集 训练数据集数据量/测试数据集数据量的比值取67/33是一个常用的惯例。
fileName = "/Users/yuhongjun/Python/python-training/data/iris.data.csv"
def loadDataset(filename, split, trainingSet=[], testSet=[]):
    with open(filename, "r", encoding="utf-8") as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        for x in range(len(dataset)-1):
            for y in range(4):
                dataset[x][y] = float(dataset[x][y])
            if random.random() < split:
                trainingSet.append(dataset[x])
            else:
                testSet.append(dataset[x])
trainingSet=[]
testSet=[]
split = 0.67
loadDataset(fileName, split,trainingSet, testSet)
print('Train set: ' + repr(len(trainingSet))) 
print('Test set: ' + repr(len(testSet)))
print('Train set: ' + repr(trainingSet))

Train set: 102
Test set: 48
Train set: [[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'], [4.6, 3.1, 1.5, 0.2, 'Iris-setosa'], [5.0, 3.6, 1.4, 0.2, 'Iris-setosa'], [5.4, 3.9, 1.7, 0.4, 'Iris-setosa'], [4.4, 2.9, 1.4, 0.2, 'Iris-setosa'], [4.9, 3.1, 1.5, 0.1, 'Iris-setosa'], [5.4, 3.7, 1.5, 0.2, 'Iris-setosa'], [4.8, 3.4, 1.6, 0.2, 'Iris-setosa'], [4.3, 3.0, 1.1, 0.1, 'Iris-setosa'], [5.8, 4.0, 1.2, 0.2, 'Iris-setosa'], [5.7, 4.4, 1.5, 0.4, 'Iris-setosa'], [5.4, 3.9, 1.3, 0.4, 'Iris-setosa'], [5.1, 3.5, 1.4, 0.3, 'Iris-setosa'], [5.7, 3.8, 1.7, 0.3, 'Iris-setosa'], [5.1, 3.8, 1.5, 0.3, 'Iris-setosa'], [5.4, 3.4, 1.7, 0.2, 'Iris-setosa'], [5.1, 3.7, 1.5, 0.4, 'Iris-setosa'], [4.6, 3.6, 1.0, 0.2, 'Iris-setosa'], [5.1, 3.3, 1.7, 0.5, 'Iris-setosa'], [5.0, 3.0, 1.6, 0.2, 'Iris-setosa'], [5.0, 3.4, 1.6, 0.4, 'Iris-setosa'], [5.2, 3.5, 1.5, 0.2, 'Iris-setosa'], [4.7, 3.2, 1.6, 0.2, 'Iris-setosa'], [4.8, 3.1, 1.6, 0.2, 'Iris-setosa'], [5.4, 3.4, 1.5, 0.4, 'Iris-setosa'], [5.5, 4.2, 1.4, 0.2, 'Iris-setosa']

In [4]:
#2、相似度
#欧式距离就是两列数中对应数做差的平方和，之后再开方
def euclideanDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += pow((instance1[x] - instance2[x]), 2)
    return math.sqrt(distance)

data1 = [5.1, 3.5, 1.4, 0.2, 'Iris-setosa']
data2 = [7.9, 3.8, 6.4, 2.0, 'Iris-virginica']
distance = euclideanDistance(data1, data2, 3)
print('Distance: ' + repr(distance))

Distance: 5.7384666941614295


In [5]:
#3、邻近元素 最直接的方法就是计算待预测数据到所有数据实例的距离，取其中距离最小的N个。

def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x],dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

trainSet = [[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'], [4.6, 3.1, 1.5, 0.2, 'Iris-setosa'],[7.9, 3.8, 6.4, 2.0, 'Iris-virginica']]
testInstance = [5,5,5]
k = 1
neighbors = getNeighbors(trainSet, testInstance, k)
print(neighbors)

[[5.1, 3.5, 1.4, 0.2, 'Iris-setosa']]


In [6]:
#4、结果 我们可以让这些近邻元素来对预测属性投票，得票最多的选项作为预测结果。
#下面这个函数实现了投票的逻辑，它假设需预测的属性放在数据实例（数组）的最后。

def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    sortedVotes = sorted(classVotes.items(), key = operator.itemgetter(1), reverse = True)
    return sortedVotes[0][0]

neighbors = [[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'], [4.6, 3.1, 1.5, 0.2, 'Iris-setosa'],[7.9, 3.8, 6.4, 2.0, 'Iris-virginica']]
response = getResponse(neighbors)
print(response)

Iris-setosa


In [7]:
#5、准确度
#一个简单的评估方法是，计算在测试数据集中算法正确预测的比例，这个比例叫分类准确度。

def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct +=1
    return(correct/float(len(testSet))) * 100.0

testSet = [[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'], [4.6, 3.1, 1.5, 0.2, 'Iris-setosa'],[7.9, 3.8, 6.4, 2.0, 'Iris-virginica']]
predictions=['Iris-setosa', 'Iris-setosa', 'Iris-setosa']
accuracy = getAccuracy(testSet, predictions)
print(accuracy)

66.66666666666666


In [ ]:
def main():
    #prepare data
    trainingSet=[]
    testSet=[]
    split = 0.67
    loadDataset(fileName, split,trainingSet, testSet)
    print('Train set: ' + repr(len(trainingSet))) 
    print('Test set: ' + repr(len(testSet)))
    # generate predictions
    predictions = []
    k = 3
    for x in range(len(testSet)):
        neighbors = getNeighbors(trainingSet, testSet[x], k)
        result = getResponse(neighbors)
        predictions.append(result)
        print(repr(x) + '> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy: ' + repr(accuracy) + '%')
main()
        

